In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/NASA_Transfer_Learning/toy')
print("Current working directory: ", os.getcwd())

Current working directory:  /content/drive/MyDrive/Colab Notebooks/NASA_Transfer_Learning/toy


In [3]:
import sys

sys.path.append("/content/drive/MyDrive/Colab Notebooks/NASA_Transfer_Learning/") # Add root directory to PATH

In [4]:
from datetime import datetime
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

from datasets import mnist, mnist_m
from models.ganin import SimpleClassifier
import trainers.ganin as trainer
# from trainer import train_ganin, test_ganin # getting called within /toy
from utils import transform, helper


# Set random seed to ensure deterministic behavior
helper.set_random_seed(seed=123)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Device:  cuda:0


In [5]:
# Install wandb ()

!pip install wandb --upgrade

     |████████████████████████████████| 2.1MB 7.0MB/s 
     |████████████████████████████████| 163kB 33.4MB/s 
     |████████████████████████████████| 102kB 10.0MB/s 
     |████████████████████████████████| 133kB 26.8MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=467b8e5fd3342f91719260492d42b3636b854445a97a0c0f5215130b9e3d4814
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=26ab5f389898ab1d991f674c220f296e9cb7064c30fa0bb2a32f15aa93ac1e77
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


In [ ]:
# Hyperparameters 
config = dict(epochs=15,
              batch_size=64,
              learning_rate=2e-4,
              classes=10,
              img_size=28,
              dataset='mnist-mnist_m',
              architecture='ganin')

In [ ]:
def main(hyperparameters):

    with wandb.init(project="uda-ganin-toy", config=hyperparameters, mode='online'):
        model = SimpleClassifier().to(device)
        wandb.watch(model, log='all', log_freq=100)

        # transforms
        transform_m = transform.get_transform(dataset="mnist")
        transform_mm = transform.get_transform(dataset="mnist_m")

        # dataloaders
        loaders_args = dict(
            batch_size=config["batch_size"],
            shuffle=True,
            num_workers=1,
            pin_memory=True,
        )

        train_src = torch.load("../data/mnist/processed/train.pt")
        trainloader_src = mnist.fetch(data=train_src,
                                    transform=transform_m,
                                    **loaders_args)

        # fetching testloader_m for symmetry but it is not needed in the code
        test_src = torch.load("../data/mnist/processed/test.pt")
        testloader_src = mnist.fetch(data=test_src,
                                transform=transform_m,
                                **loaders_args)

        train_tgt = torch.load("../data/mnist_m/processed/train.pt")
        trainloader_tgt = mnist_m.fetch(data=train_tgt,
                                    transform=transform_mm,
                                    **loaders_args)

        test_tgt = torch.load("../data/mnist_m/processed/test.pt")
        testloader_tgt = mnist_m.fetch(data=test_tgt,
                                    transform=transform_mm,
                                    **loaders_args)
        

        criterion = nn.CrossEntropyLoss() # class labels 1 to N class
        optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
        
        start_time = datetime.now()
        for epoch in range(1, config["epochs"]+1):
            
            loss = trainer.train_simple_classifier(model, epoch, config, trainloader_src,
                        criterion, optimizer, device)
            
            wandb.log({"epoch" : epoch, "train_loss" : loss}, step=epoch)

            accuracy = trainer.test_simple_classifier(model, testloader_tgt, device)

            wandb.log({"epoch" : epoch, "val_accuracy" : accuracy}, step=epoch)

        end_time = datetime.now()
        print(f"Train Time for {config['epochs']} epochs: {end_time - start_time}")

    return model

In [ ]:
output = main(config)